In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 28 19:55:26 2019

@author: ericl
"""

import os
import librosa
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import pandas as pd
from tqdm import tqdm

#These two dir are used to track the name of the files we want to convert, as only 200 test audios were selected, we track them from
# the ones generated with matlab
clean_train_folder = r'F:\ECE271B_Project\Noise_Addition\timit_128\timit\train'
clean_test_folder=r'F:\ECE271B_Project\Noise_Addition\results\test\babble\0db'

#These are the folders where we have our noisy data stored
noisy_test_folder = r'F:\ECE271B_Project\Noise_Addition\results\test'
noisy_train_folder = r'F:\ECE271B_Project\Noise_Addition\results\train'

#output stft features in numpy form and save in below dirs
output_test_folder=r'F:\testset_results_data'
output_train_folder=r'F:\trainset_results_data'

#the type of noise and SNR we want to deal with, add in dirs to ensure consistency
noisy_types=[r'\babble',r'\white',r'\factory1',r'\hfchannel']
SNRs=[r'\5db',r'\10db',r'\15db',r'\20db',r'\0db',r'\-5db']
#SNRs=[r'\5db']
#noisy_types=[r'\babble']


window_size = 2 ** 14  # about 1 second of samples
#sample_rate = 16000


def saveConvert_info(file):
    """
    input a wav file, return np array after stft
    """
    
    y, fs = librosa.load(file, sr=8000)
    D = librosa.core.stft(y, n_fft = 128)
    #sample_rate, samples = wavfile.read(file)
    #x=scipy.signal.stft(samples,sample_rate)
    D_a = np.abs(D)
    D_db = librosa.core.amplitude_to_db(D_a, ref=np.max)
    phase=np.angle(D)
    mean=np.mean(D_db,1)
    std=np.std(D_db,1)
    max_value=np.max(D_a)
    return [phase,mean,std,max_value]

def saveConvert_data(file):
    """
    input a wav file, return np array after stft
    """
    sample_rate, samples = wavfile.read(file)
    x=scipy.signal.stft(samples,sample_rate)
    D_a = np.abs(x[2])

    D_db = librosa.core.amplitude_to_db(D_a, ref=np.max)
    return D_db
    

def normalize(data):
    """
    normalize data by each row
    
    intype: np array 129*n
    rtype: np array 129*n
    
    """
    #this function should not be utilized until we get the mean and std of our data
    return (data-np.mean(data,axis=1).reshape(129,1))/np.std(data,axis=1).reshape(129,1)

test_dict={}
def processData(data_type):
    """
    Serialize, down-sample the sliced signals and save on separate folder.
    """
    for snr in SNRs:
        for noise in noisy_types:
            #max_idxs=[]
            
            if data_type == 'train':
                clean_folder = clean_train_folder
                noisy_folder = noisy_train_folder+noise+snr
                serialized_folder = serialized_train_folder+noise+snr
            else:
                clean_folder = clean_test_folder
                noisy_folder = noisy_test_folder+noise+snr
                serialized_folder = serialized_test_folder+noise+snr
            if not os.path.exists(serialized_folder):
                os.makedirs(serialized_folder)
            
            #clean_folder = clean_test_folder
            #noisy_folder = noisy_test_folder+noise+snr
            
            """
            #this piece of code is used to generate converted data along with their phases,angle,etc.
            for root, dirs, files in os.walk(clean_folder):
                if len(files) == 0:
                    continue
                #print('current folder',dirs)
                for filename in tqdm(files, desc='Converting {} audios'.format(data_type)):
                    noisy_file = os.path.join(noisy_folder, filename)
                    if '.wav' in filename:
                    #[phase,mean,std,max_value]


                        data=saveConvert_data(noisy_file)
                        data=normalize(data)
                        (a,b,c,d)=saveConvert_info(noisy_file)
                        test_dict['phase']=a
                        test_dict['mean']=b
                        test_dict['std']=c
                        test_dict['max_value']=d
                        test_dict['data']=data
                        np.save(os.path.join(serialized_folder, '{}'.format(filename)),test_dict)
                        #np.save(os.path.join(serialized_folder, '{}'.format(filename)), arr=test_dict)
                        #print(noisy_file)

                        
            """
            
            for root, dirs, files in os.walk(clean_folder):
                for filename in tqdm(files, desc='Converting {} audios'.format(data_type)):
                    if '.wav' in filename:
                        noisy_file = os.path.join(noisy_folder, filename)
                        converted_noisy=saveConvert_data(noisy_file)
                        #test=normalize(converted_noisy)
                        np.save(os.path.join(serialized_folder, '{}'.format(filename)), arr=test)
            

                    #print(np.sum(np.isnan(converted_noisy)))
                    #max_idxs.append((filename,converted_noisy.shape[1]))
                    
            #print(serialized_folder)
            
    

In [116]:
process_and_serialize('train')


Converting train audios:  25%|█████████████▏                                       | 1153/4621 [00:14<00:43, 78.91it/s]


Converting train audios:  51%|███████████████████████████                          | 2361/4621 [00:29<00:27, 81.48it/s]


Converting train audios:  76%|████████████████████████████████████████▎            | 3517/4621 [00:44<00:13, 81.03it/s]


Converting train audios:   2%|▊                                                      | 72/4621 [00:00<00:55, 82.52it/s]


Converting train audios:  28%|██████████████▋                                      | 1281/4621 [00:16<00:40, 81.65it/s]


...

Converting train audios:  17%|█████████▎                                            | 801/4621 [00:10<00:49, 77.51it/s]


Converting train audios:  43%|██████████████████████▊                              | 1984/4621 [00:25<00:33, 77.60it/s]


Converting train audios:  68%|████████████████████████████████████                 | 3141/4621 [00:40<00:19, 75.40it/s]


Converting train audios:  94%|█████████████████████████████████████████████████▌   | 4323/4621 [00:55<00:03, 77.53it/s]


Converting train audios: 100%|█████████████████████████████████████████████████████| 4621/4621 [00:59<00:00, 78.20it/s]

In [95]:
data=np.array([])

In [ ]:
SNRs=

In [2]:
def get_std(data_type):
    """
    Serialize, down-sample the sliced signals and save on separate folder.
    """
    data=np.array([])
    for snr in SNRs:
        for noise in noisy_types:
            #max_idxs=[]
            
            if data_type == 'train':
                clean_folder = clean_train_folder
                noisy_folder = noisy_train_folder+noise+snr
                serialized_folder = serialized_train_folder+noise+snr
            else:
                clean_folder = clean_test_folder
                noisy_folder = noisy_test_folder+noise+snr
                serialized_folder = serialized_test_folder+noise+snr
            if not os.path.exists(serialized_folder):
                os.makedirs(serialized_folder)
            
            #clean_folder = clean_test_folder
            #noisy_folder = noisy_test_folder+noise+snr
            phase_max=0
            phase_min=0
            
            for root, dirs, files in os.walk(clean_folder):
                if len(files) == 0:
                    continue
                #print('current folder',dirs)
                for filename in tqdm(files, desc='Converting {} audios'.format(data_type)):
                    noisy_file = os.path.join(noisy_folder, filename)
                    if '.wav' in filename:
                    #[phase,mean,std,max_value]
                        if len(data)==0:
                            data=saveConvert_data(noisy_file)
                        else:
                            data=np.hstack((data,saveConvert_data(noisy_file)))



    return data


In [97]:
data=get_std('s')

Converting s audios: 100%|███████████████████████████████████████████████████████████| 201/201 [01:02<00:00,  3.20it/s]


In [ ]:
peng

In [98]:
data.shape

(129, 1364328)

In [100]:
np.save('temp_data_overall.npy',data)

In [101]:
test_mean=np.mean(data,axis=1)

In [102]:
test_mean

array([-34.96169 , -33.29848 , -34.88974 , -32.768406, -31.24605 ,
       -30.986795, -30.389713, -29.464787, -29.23138 , -30.241167,
       -30.691772, -30.180916, -29.130016, -28.24436 , -28.468357,
       -28.78434 , -28.924969, -29.047962, -29.363771, -29.972277,
       -30.190529, -30.516947, -31.05108 , -31.501698, -31.857124,
       -32.059032, -32.32082 , -32.953323, -33.36739 , -33.745277,
       -34.08332 , -34.41312 , -34.64719 , -34.857655, -35.025757,
       -35.138535, -35.314865, -35.552727, -35.64866 , -35.714207,
       -35.855446, -36.049732, -36.104492, -36.14322 , -36.158287,
       -36.173336, -36.22266 , -35.974583, -34.94144 , -34.640594,
       -35.678677, -36.40337 , -36.59329 , -36.746883, -36.810577,
       -36.896076, -36.98276 , -37.093754, -37.234318, -37.32701 ,
       -37.502808, -37.563282, -37.568443, -37.514107, -37.418285,
       -37.409687, -37.395542, -37.76427 , -38.135853, -38.192867,
       -38.199326, -38.172226, -38.08501 , -37.721344, -36.997

In [103]:
test_std=np.std(data,axis=1)

In [104]:
test_std

array([14.028719 , 11.699112 , 11.206689 , 10.299333 , 10.354731 ,
       10.202937 , 10.0142   , 10.190651 , 10.467332 , 10.435044 ,
       10.50734  , 10.694622 , 10.77027  , 11.025261 , 11.283506 ,
       11.562296 , 11.581502 , 11.541246 , 11.541222 , 11.633279 ,
       11.639541 , 11.551707 , 11.426256 , 11.32813  , 11.198998 ,
       11.037219 , 10.963775 , 10.902885 , 10.796595 , 10.773562 ,
       10.82782  , 10.8588295, 10.801889 , 10.742018 , 10.738283 ,
       10.743392 , 10.791183 , 10.801586 , 10.743846 , 10.705947 ,
       10.674794 , 10.688099 , 10.677687 , 10.66166  , 10.61488  ,
       10.596549 , 10.591666 , 10.6810465, 11.071444 , 11.21365  ,
       10.757053 , 10.586722 , 10.62622  , 10.65938  , 10.65983  ,
       10.638254 , 10.605152 , 10.606868 , 10.613156 , 10.646145 ,
       10.667331 , 10.696832 , 10.671836 , 10.674796 , 10.673907 ,
       10.663479 , 10.708823 , 10.651448 , 10.588862 , 10.542684 ,
       10.557628 , 10.593143 , 10.582206 , 10.683856 , 10.9390

In [88]:
np.max(data,axis=1)

array([ 3.8146973e-06, -5.0424042e+00, -1.0679150e+01, -1.0377800e+01,
       -5.1098557e+00, -5.9115601e+00, -7.2055969e+00, -4.4684525e+00,
       -4.3663559e+00, -2.8498116e+00, -1.4882317e+00,  3.8146973e-06,
       -3.8146973e-06,  3.8146973e-06,  3.8146973e-06,  3.8146973e-06,
        0.0000000e+00,  3.8146973e-06,  3.8146973e-06,  3.8146973e-06,
        3.8146973e-06,  3.8146973e-06,  3.8146973e-06,  0.0000000e+00,
        3.8146973e-06,  3.8146973e-06,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  3.8146973e-06,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -4.6288681e-01, -8.1175995e-01, -9.6710205e-02,
       -8.5424042e-01, -1.0558586e+00, -6.5783691e-01, -1.5104446e+00,
       -1.6981163e+00, -3.6734352e+00, -3.9402313e+00, -3.7969933e+00,
       -1.6625862e+00, -4.5643349e+00, -4.6465378e+00, -3.1866531e+00,
       -3.3954849e+00, -3.8848610e+00, -3.1499596e+00, -3.3743896e+00,
       -5.0973549e+00, -3.4245033e+00, -4.8213348e+00, -1.5562134e+00,
      

In [58]:
test=np.zeros((129,256))

In [66]:
newstack=np.hstack((test,test))

In [67]:
newstack.shape

(129, 512)

In [38]:
np.stack?

In [ ]:
np.angle